In [3]:
from scipy.io import arff
import pandas as pd
import numpy as np

In [4]:
data= arff.loadarff('4year.arff')
df=pd.DataFrame(data[0])

In [5]:
df['bankruptcy']=(df['class']==b'1')

In [6]:
df.head()

,Attr1,Attr2,Attr3,Attr4,Attr5,Attr6,Attr7,Attr8,Attr9,Attr10,...,Attr57,Attr58,Attr59,Attr60,Attr61,Attr62,Attr63,Attr64,class,bankruptcy
0,0.159290,0.46240,0.07773,1.1683,-44.853,0.467020,0.189480,0.82895,1.1223,0.38330,...,0.41557,0.89101,0.001422,7.7928,4.9914,119.810,3.0465,3.0560,b'0',False
1,-0.127430,0.46243,0.26917,1.7517,7.597,0.000925,-0.127430,1.16250,1.2944,0.53757,...,-0.23704,1.06250,0.150410,5.4327,3.4629,100.970,3.6150,3.4725,b'0',False
2,0.070488,0.23570,0.52781,3.2393,125.680,0.163670,0.086895,2.87180,1.0574,0.67689,...,0.10413,0.94571,0.000000,7.1070,3.3808,76.076,4.7978,4.7818,b'0',False
3,0.136760,0.40538,0.31543,1.8705,19.115,0.504970,0.136760,1.45390,1.1144,0.58938,...,0.23203,0.89737,0.073024,6.1384,4.2241,88.299,4.1337,4.6484,b'0',False
4,-0.110080,0.69793,0.18878,1.2713,-15.344,0.000000,-0.110080,0.43282,1.7350,0.30207,...,-0.36440,0.57153,0.000000,18.8010,2.7925,146.390,2.4934,15.0360,b'0',False


# 1.Create a new dataframe with only 4 feataures and Bankruptcy. Properly rename the columns

In [7]:
df1=df.loc[:,['Attr1','Attr2','Attr7','Attr10','bankruptcy']]
df1.head()

,Attr1,Attr2,Attr7,Attr10,bankruptcy
0,0.159290,0.46240,0.189480,0.38330,False
1,-0.127430,0.46243,-0.127430,0.53757,False
2,0.070488,0.23570,0.086895,0.67689,False
3,0.136760,0.40538,0.136760,0.58938,False
4,-0.110080,0.69793,-0.110080,0.30207,False


# 2.Fill-in the missing values na with the mean

In [159]:
df1[df1.isnull().values==1]

,X1,X2,X7,X10,bankruptcy
2898,NaN,NaN,NaN,NaN,False
2898,NaN,NaN,NaN,NaN,False
2898,NaN,NaN,NaN,NaN,False
2898,NaN,NaN,NaN,NaN,False


In [141]:
from sklearn.impute import SimpleImputer
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean = imr.fit(df1.values)
imputed_data = imp_mean.transform(df1.values)
imputed_data

array([[ 0.15929 ,  0.4624  ,  0.18948 ,  0.3833  ,  0.      ],
       [-0.12743 ,  0.46243 , -0.12743 ,  0.53757 ,  0.      ],
       [ 0.070488,  0.2357  ,  0.086895,  0.67689 ,  0.      ],
       ...,
       [-0.23829 ,  0.62708 , -0.24036 ,  0.1776  ,  1.      ],
       [ 0.097188,  0.753   ,  0.10428 ,  0.247   ,  1.      ],
       [ 0.021416,  0.48678 ,  0.027253,  0.51267 ,  1.      ]])

In [154]:
imputed_data[2898:2899]

array([[0.04301917, 0.59640382, 0.0594459 , 0.38904008, 0.        ]])

In [153]:
df1.fillna(df1.mean(axis=0)[0:4])
df1[2898:2899]
#this method does not work well

,X1,X2,X7,X10,bankruptcy
2898,NaN,NaN,NaN,NaN,False


# 3.Find the mean and std of the 4 features among all, bankrupt and still-operating companies (3 groups).

In [76]:
dft=df1[df1['bankruptcy']>0]
dff=df1[df1['bankruptcy']==0]
result1=np.array([df1.mean(axis=0)[0:4],df1.std(axis=0)[0:4]])
result1.reshape(2,4)
result2=np.array([dft.mean(axis=0)[0:4],dft.std(axis=0)[0:4]])
result2.reshape(2,4)
result3=np.array([dff.mean(axis=0)[0:4],dff.std(axis=0)[0:4]])
result3.reshape(2,4)
print(result1)
print(result2)
print(result3)

[[0.04301917 0.59640382 0.0594459  0.38904008]
 [0.35932092 4.58712166 0.53334393 4.59029866]]
[[-0.06887268  0.87835495 -0.06153788  0.1033668 ]
 [ 0.56807576  1.94559616  0.56843199  1.94674701]]
[[0.04923136 0.58075    0.06616287 0.40490056]
 [0.34302023 4.689947   0.53055269 4.69318723]]


# 4.How many companies have X1 values 1 std below the mean AND X10 values 1 std below the mean?

In [120]:
dfx1=df1[df1['X1']<df1.mean(axis=0)['X1']-df1.std(axis=0)['X1']]
dfx1_10=df1[df1['X10']<df1.mean(axis=0)['X10']-df1.std(axis=0)['X10']]
dfx1_10.shape[0]
print('There are %d companies satisfy X1 < mean(X1) - stdev(X1) AND X10 < mean(X10) - std(X10)'%dfx1_10.shape[0])
# pay attention to the output number in print function

There are 24 companies satisfy X1 < mean(X1) - stdev(X1) AND X10 < mean(X10) - std(X10)


# 5.What is the ratio of the bankrupted companies among the sub-groups above?

In [127]:
temp=dfx1_10[dfx1_10['bankruptcy']==0]
ratio=temp.shape[0]/dfx1_10.shape[0]
print(ratio)

0.875
